In [126]:
import pyGeno as pg
import pandas as pd
import os

In [127]:
from Bio import AlignIO 
from Bio import SeqIO

In [156]:
gbfile = '/data/EBV/byACCIDs/genBankRecord_NC_007605.gb'
genome = SeqIO.read(gbfile,'genbank')
path = '/data/EBV/refGenes/'

if not os.path.isdir(path):
    os.system('mkdir %s'%path)

thr = 50

In [157]:
gene_coords_fromNCBI = pd.read_csv('/home/ariel/Downloads/gene_result.txt',sep='\t',
                                   usecols=['GeneID','Symbol','Aliases','description',
                                            'start_position_on_the_genomic_accession',
                                            'end_position_on_the_genomic_accession'])

gene_coords_fromNCBI['length'] =gene_coords_fromNCBI['end_position_on_the_genomic_accession']-\
                                gene_coords_fromNCBI['start_position_on_the_genomic_accession']

    
short_seqs = gene_coords_fromNCBI[gene_coords_fromNCBI.length<=thr]
long_seqs = gene_coords_fromNCBI[gene_coords_fromNCBI.length>thr]
print 'There are %s genes'%long_seqs.shape[0]


stcol = 'start_position_on_the_genomic_accession' 
endcol = 'end_position_on_the_genomic_accession'        

#verifico si perdi alguno
short_seqs[~short_seqs.Symbol.isin(long_seqs.Symbol)]



There are 104 genes


,GeneID,Symbol,Aliases,description,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,length
16,17494233,BBRF1,NaN,NaN,101588.0,101593.0,5.0
17,17494232,BBRF1,NaN,NaN,104497.0,104502.0,5.0
24,17494236,BCR2,NaN,NaN,11305.0,11308.0,3.0
55,17494241,BGRF1-BDRF1,NaN,NaN,118059.0,118064.0,5.0
56,17494251,BHF1,NaN,NaN,41471.0,41476.0,5.0
67,17494189,BLL3,NaN,NaN,75320.0,75325.0,5.0
122,17494198,EBNA-3C,NaN,NaN,89477.0,89482.0,5.0
127,17494216,EBNA3A,NaN,NaN,82933.0,82938.0,5.0
139,17494222,lytic LMP1,NaN,NaN,168738.0,168743.0,5.0
140,17494253,miBART-2,microRNA BART-2,miscRNA,152747.0,152768.0,21.0


In [158]:
long_seqs[long_seqs.Symbol.str.contains('EBNA')]

,GeneID,Symbol,Aliases,description,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,length
118,17494235,EBNA,NaN,NaN,11336.0,11480.0,144.0
119,17494227,EBNA-1,NaN,NaN,55189.0,55361.0,172.0
121,17494192,EBNA-2,NaN,NaN,36098.0,37739.0,1641.0
123,3783746,EBNA-LP,NaN,EBNA-LP protein,14352.0,35711.0,21359.0
124,17494199,EBNA-LP,NaN,NaN,36098.0,36156.0,58.0
125,17494195,EBNA-LP,NaN,NaN,37564.0,37744.0,180.0
126,17494201,EBNA1 leader sequence,NaN,NaN,86076.0,86442.0,366.0
128,3783763,EBNA3B,NaN,EBNA-3B nuclear protein,83065.0,89502.0,6437.0
129,17494242,EBNA3C,NaN,EBNA3C latent protein,86076.0,89502.0,3426.0


In [159]:
%matplotlib inline
table = gene_coords_fromNCBI.length.value_counts().sort_index()#.plot(kind='density',loglog=True)
table = table[table.index <160000]
table.head(10)
#table
#table.hist(bins = 100)
#table.plot(loglog=True).
#.plot(kind='scatter',logx=True,logy=True)

0.0      2
3.0      1
4.0      2
5.0     35
6.0      1
7.0      1
21.0     3
22.0     2
39.0     1
58.0     1
Name: length, dtype: int64

## Write to disk each gene sequence

In [145]:
for i in long_seqs.index:
    symbol = long_seqs.Symbol[i]
    Id = long_seqs.GeneID[i]
    with open(path+'GId:'+str(Id)+'-'+symbol+'.fa','w') as of:
        st = int(long_seqs[stcol][i])-1
        end = int(long_seqs[endcol][i])
        gf = genome.seq[st:end].tostring()
        of.write(">%s|GeneId %s from NC_007605\n%s\n"%(symbol,Id,gf))



In [ ]:
if (False):
    seqs ={}
    wanted = ['BKRF1','EBNA3A']

    for g in wanted:
        outputfile = '/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/bygene/%s_test.fa'%g

        with open(outputfile, 'w') as ofile:
            for record in SeqIO.parse(gbfile, "genbank"):
                for f in record.features:
                    if f.type == "CDS" and "gene" in f.qualifiers:
                    #if 'gene' in f.qualifiers:
                        gene = f.qualifiers["gene"][0]
                        #print gene
                        if gene  in wanted:
                            ofile.write(">%s|%s from %s\n%s\n%s\n" %(f.qualifiers['gene'][0],
                                                                     f.qualifiers['db_xref'].
                                                                 f.qualifiers['product'][0],
                                                                 record.name,
                                                                 f.location.extract(record).seq,
                                                                 f.location ))

            ofile.close()